In [ ]:
# Função para otimizar o uso de memória
def reduce_memory_usage(df):
    # Iterar sobre todas as colunas do DataFrame
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            elif str(col_type)[:5] == 'float':
                df[col] = df[col].astype(np.float32)
        else:
            # Convert object types to category
            if df[col].nunique() < 0.5 * len(df):
                df[col] = df[col].astype('category')

    return df

# Ligar com user_id
def generate_user_features(data):
    # Selecionar apenas as colunas necessárias do DataFrame de entrada
    temp_data = data[['user_id', 'order_id', 'product_id', 'reordered', 'days_since_prior_order']].copy()

    # Calcular o número total de pedidos por usuário
    user_order_count = temp_data.groupby('user_id')['order_id'].nunique().reset_index()
    user_order_count.columns = ['user_id', 'user_order_count']

    # Calcular a média de produtos por pedido por usuário
    user_avg_products_per_order = temp_data.groupby('user_id')['product_id'].count() / temp_data.groupby('user_id')['order_id'].nunique()
    user_avg_products_per_order = user_avg_products_per_order.reset_index(name='user_avg_products_per_order')

    # Calcular a diversidade de produtos comprados por usuário
    user_unique_products = temp_data.groupby('user_id')['product_id'].nunique().reset_index()
    user_unique_products.columns = ['user_id', 'user_unique_products']

    # Calcular a frequência de reordem por usuário
    user_reorder_ratio = temp_data.groupby('user_id')['reordered'].mean().reset_index()
    user_reorder_ratio.columns = ['user_id', 'user_reorder_ratio']

    # Calcular a média de dias entre pedidos
    user_avg_days_between_orders = temp_data.groupby('user_id')['days_since_prior_order'].mean().reset_index()
    user_avg_days_between_orders.columns = ['user_id', 'user_avg_days_between_orders']

    # Combinar todas as features em um único DataFrame
    user_features = user_order_count.merge(user_avg_products_per_order, on='user_id')
    user_features = user_features.merge(user_unique_products, on='user_id')
    user_features = user_features.merge(user_reorder_ratio, on='user_id')
    user_features = user_features.merge(user_avg_days_between_orders, on='user_id')

    # Retornar o DataFrame final de features por usuário
    return user_features

def generate_purchase_features_antiga(data):
    # Selecionar apenas as colunas necessárias do DataFrame de entrada
    temp_data = data[['product_id', 'aisle_id', 'department_id', 'order_id', 'reordered', 'add_to_cart_order']].copy()

    # Calcular a frequência de compra por produto
    product_freq = temp_data.groupby('product_id')['order_id'].count().reset_index()
    product_freq.columns = ['product_id', 'product_order_freq']

    # Calcular a proporção de reordem por produto
    product_reorder_ratio = temp_data.groupby('product_id')['reordered'].mean().reset_index()
    product_reorder_ratio.columns = ['product_id', 'product_reorder_ratio']

    # Calcular a posição média do produto no carrinho
    avg_pos_incart = temp_data.groupby('product_id')['add_to_cart_order'].mean().reset_index()
    avg_pos_incart.columns = ['product_id', 'avg_pos_incart']

    # Combinar a frequência de compra e a proporção de reordem em um único DataFrame de features de produto
    product_features = product_freq.merge(product_reorder_ratio, on='product_id')
    product_features = product_features.merge(avg_pos_incart, on='product_id')

    # Calcular a frequência de compra por corredor
    aisle_freq = temp_data.groupby('aisle_id')['order_id'].count().reset_index()
    aisle_freq.columns = ['aisle_id', 'aisle_order_freq']

    # Calcular a proporção de reordem por corredor
    aisle_reorder_ratio = temp_data.groupby('aisle_id')['reordered'].mean().reset_index()
    aisle_reorder_ratio.columns = ['aisle_id', 'aisle_reorder_ratio']

    # Combinar a frequência de compra e a proporção de reordem em um único DataFrame de features de corredor
    aisle_features = pd.merge(aisle_freq, aisle_reorder_ratio, on='aisle_id')

    # Calcular a frequência de compra por departamento
    department_freq = temp_data.groupby('department_id')['order_id'].count().reset_index()
    department_freq.columns = ['department_id', 'department_order_freq']

    # Calcular a proporção de reordem por departamento
    department_reorder_ratio = temp_data.groupby('department_id')['reordered'].mean().reset_index()
    department_reorder_ratio.columns = ['department_id', 'department_reorder_ratio']

    # Combinar a frequência de compra e a proporção de reordem em um único DataFrame de features de departamento
    department_features = pd.merge(department_freq, department_reorder_ratio, on='department_id')

    # Combinar todas as features em um único DataFrame de features de compra
    unique_products = data[['product_id', 'aisle_id', 'department_id']].drop_duplicates()
    features = product_features.merge(unique_products, on='product_id', how='left')
    features = features.merge(aisle_features, on='aisle_id', how='left')
    features = features.merge(department_features, on='department_id', how='left')

    return features

# Ligar com user_id
def generate_temporal_features(data):
    # Selecionar apenas as colunas necessárias do DataFrame de entrada e remover duplicatas
    temp_data = data[['user_id', 'order_hour_of_day', 'order_dow']].drop_duplicates().copy()

    # Calcular a representação cíclica para o dia da semana (order_dow)
    temp_data['order_dow_sin'] = np.sin(2 * np.pi * temp_data['order_dow'] / 7).astype(np.float32)
    temp_data['order_dow_cos'] = np.cos(2 * np.pi * temp_data['order_dow'] / 7).astype(np.float32)

    # Calcular a representação cíclica para a hora do dia (order_hour_of_day)
    temp_data['order_hour_of_day_sin'] = np.sin(2 * np.pi * temp_data['order_hour_of_day'] / 24).astype(np.float32)
    temp_data['order_hour_of_day_cos'] = np.cos(2 * np.pi * temp_data['order_hour_of_day'] / 24).astype(np.float32)

    # Calcular a frequência de pedidos por hora do dia para cada usuário
    user_hour_order_freq = data.groupby(['user_id', 'order_hour_of_day']).size().reset_index(name='user_hour_order_freq')

    # Calcular a frequência de pedidos por dia da semana para cada usuário
    user_dow_order_freq = data.groupby(['user_id', 'order_dow']).size().reset_index(name='user_dow_order_freq')

    # Combinar as frequências calculadas com os dados temporais
    temp_data = temp_data.merge(user_hour_order_freq, on=['user_id', 'order_hour_of_day'], how='left')
    temp_data = temp_data.merge(user_dow_order_freq, on=['user_id', 'order_dow'], how='left')

    # Remover as colunas 'order_dow' e 'order_hour_of_day'
    temp_data = temp_data.drop(columns=['order_dow', 'order_hour_of_day'])

    # Remover duplicatas, mantendo apenas uma entrada por usuário
    temporal_features = temp_data.drop_duplicates(subset=['user_id']).reset_index(drop=True)

    # Retornar o DataFrame final de features temporais
    return temporal_features

# test_data = test_data.merge(user_session_features, on='user_id', how='left')
# test_data = test_data.merge(product_days_diff, on='product_id', how='left')
def generate_session_features(data):
    # Selecionar apenas as colunas necessárias do DataFrame de entrada
    temp_data = data[['user_id', 'order_number', 'days_since_prior_order']].copy()

    # Calcular a média de dias desde o pedido anterior por usuário
    user_avg_days_since_prior = temp_data.groupby('user_id')['days_since_prior_order'].mean().reset_index()
    user_avg_days_since_prior.columns = ['user_id', 'user_avg_days_since_prior']

    # Calcular o número de dias desde o último pedido por usuário
    user_days_since_last_order = temp_data.groupby('user_id')['days_since_prior_order'].last().reset_index()
    user_days_since_last_order.columns = ['user_id', 'user_days_since_last_order']

    # Combinar as duas features de usuário em um único DataFrame
    user_session_features = user_avg_days_since_prior.merge(user_days_since_last_order, on='user_id')

    # Calcular a diferença média entre pedidos que contêm o mesmo produto
    product_days_diff = data.groupby('product_id')['days_since_prior_order'].mean().reset_index()
    product_days_diff.columns = ['product_id', 'product_avg_days_since_prior']

    # Retornar os DataFrames de features de usuário e de produto
    return user_session_features, product_days_diff

# Ligar com user_id
def generate_behavior_features(data):
    # Selecionar apenas as colunas necessárias do DataFrame de entrada
    temp_data = data[['user_id', 'reordered', 'order_id']].copy()

    # Calcular a taxa de recompra por usuário
    user_reorder_rate = temp_data.groupby('user_id')['reordered'].mean().reset_index()
    user_reorder_rate.columns = ['user_id', 'user_reorder_rate']

    # Calcular o número total de pedidos por usuário
    user_order_count = temp_data.groupby('user_id')['order_id'].count().reset_index()
    user_order_count.columns = ['user_id', 'user_order_count']

    # Calcular a frequência de recompra por usuário
    user_reorder_freq = temp_data.groupby('user_id')['reordered'].sum().reset_index()
    user_reorder_freq.columns = ['user_id', 'user_reorder_freq']

    # Calcular a proporção de reordens por usuário
    user_reordered_products_ratio = temp_data.groupby('user_id')['reordered'].mean().reset_index()
    user_reordered_products_ratio.columns = ['user_id', 'user_reordered_products_ratio']

    # Combinar todas as features de comportamento em um único DataFrame
    behavior_features = user_reorder_rate.merge(user_order_count, on='user_id')
    behavior_features = behavior_features.merge(user_reorder_freq, on='user_id')
    behavior_features = behavior_features.merge(user_reordered_products_ratio, on='user_id')

    # Remover a coluna user_order_count antes de retornar o DataFrame final
    behavior_features = behavior_features.drop(columns=['user_order_count'])

    # Retornar o DataFrame final de features de comportamento
    return behavior_features

def generate_product_features_2(data):
    # Selecionar apenas as colunas necessárias do DataFrame de entrada e remover duplicatas
    temp_data = data[['product_id', 'user_id', 'order_id', 'add_to_cart_order', 'reordered']].drop_duplicates().copy()

    # Calcular a popularidade do produto (total de pedidos para cada produto)
    product_popularity = temp_data.groupby('product_id')['order_id'].count().reset_index()
    product_popularity.columns = ['product_id', 'product_popularity']

    # Calcular a popularidade do produto por usuário (número de pedidos por produto por usuário)
    user_product_popularity = temp_data.groupby(['product_id', 'user_id'])['order_id'].count().reset_index()
    user_product_popularity.columns = ['product_id', 'user_id', 'user_product_popularity']

    # Agregar a popularidade do produto por usuário (soma total das popularidades)
    user_product_popularity = user_product_popularity.groupby('product_id')['user_product_popularity'].sum().reset_index()

    # Calcular a posição média dos produtos nos pedidos
    avg_pos_incart = temp_data.groupby('product_id')['add_to_cart_order'].mean().reset_index()
    avg_pos_incart.columns = ['product_id', 'avg_pos_incart']

    # Combinar as features de popularidade do produto em um único DataFrame
    features = product_popularity.merge(user_product_popularity, on='product_id', how='left')
    features = features.merge(avg_pos_incart, on='product_id', how='left')

    # Remover a coluna avg_pos_incart antes de retornar o DataFrame final
    features = features.drop(columns=['avg_pos_incart'])

    # Retornar o DataFrame final de features de produto
    return features

# ----------------------------------------- Novas funções --------------------------------------

def generate_product_features(data):
    # Selecionar apenas as colunas necessárias do DataFrame de entrada e remover duplicatas
    temp_data = data[['product_id', 'user_id', 'order_id', 'reordered']].drop_duplicates().copy()

    # Calcular a popularidade do produto (total de pedidos para cada produto)
    product_popularity = temp_data.groupby('product_id')['order_id'].count().reset_index()
    product_popularity.columns = ['product_id', 'product_popularity']

    # Calcular a popularidade do produto por usuário (número de pedidos por produto por usuário)
    user_product_popularity = temp_data.groupby(['product_id', 'user_id'])['order_id'].count().reset_index()
    user_product_popularity.columns = ['product_id', 'user_id', 'user_product_popularity']

    # Agregar a popularidade do produto por usuário (soma total das popularidades)
    user_product_popularity = user_product_popularity.groupby(['product_id', 'user_id'])['user_product_popularity'].sum().reset_index()

    # Combinar as features de popularidade do produto em um único DataFrame
    features = product_popularity.merge(user_product_popularity, on='product_id', how='left')

    # Retornar o DataFrame final de features de produto com user_id
    return features

# Ligar com product_id
def generate_purchase_features(data):
    # Selecionar apenas as colunas necessárias do DataFrame de entrada
    temp_data = data[['product_id', 'aisle_id', 'department_id', 'order_id', 'reordered', 'add_to_cart_order']].copy()

    # Calcular a frequência de compra por produto
    product_freq = temp_data.groupby('product_id')['order_id'].count().reset_index()
    product_freq.columns = ['product_id', 'product_order_freq']

    # Calcular a proporção de reordem por produto
    product_reorder_ratio = temp_data.groupby('product_id')['reordered'].mean().reset_index()
    product_reorder_ratio.columns = ['product_id', 'product_reorder_ratio']

    # Calcular a posição média do produto no carrinho
    avg_pos_incart = temp_data.groupby('product_id')['add_to_cart_order'].mean().reset_index()
    avg_pos_incart.columns = ['product_id', 'avg_pos_incart']

    # Combinar a frequência de compra e a proporção de reordem em um único DataFrame de features de produto
    product_features = product_freq.merge(product_reorder_ratio, on='product_id')
    product_features = product_features.merge(avg_pos_incart, on='product_id')

    # Retornar o DataFrame final de features de produto
    return product_features
